In [1]:
from IPython.display import clear_output
clear_output
from random import shuffle, randint
suits = ['Diamonds', 'Hearts', 'Clubs', 'Spades']
ranks = ['Two', 'Three', 'Four', 'Five', 'Six', 'Seven', 'Eight', 'Nine', 'Ten',
        'Jack', 'Queen', 'King', 'Ace']
pairs = {'Two': 2, 'Three': 3, 'Four': 4, 'Five': 5, 'Six': 6, 'Seven': 7, 'Eight': 8,
        'Nine': 9, 'Ten': 10, 'Jack': 10, 'Queen': 10, 'King': 10, 'Ace': 11 }

In [2]:
class Deck():
    
    def __init__(self):
        self.all_cards = []
        for suit in suits:
            for rank in ranks:
                created_card = Card(rank, suit)
                self.all_cards.append(created_card)
    def shuffle(self): 
        shuffle(self.all_cards)
        
    def deal_one(self):
        return self.all_cards.pop()
    def deal_two(self):
        popped = list(self.all_cards.pop() for _ in range(2))
        return popped
    def __len__(self):
        return len(self.all_cards)

In [3]:
class Card:
    def __init__(self, rank, suit):
        self.suit = suit
        self.rank = rank
        self.pairs = pairs[rank]
    
    def __str__(self):
        return self.rank + ' of ' + self.suit 
        

In [4]:
class Hand():
    def __str__(self):
        return f'Player {self.name} has {len(self.all_cards)} cards'
    
    def add_card(self, new_cards):
        if type(new_cards) == type([]):
            self.all_cards.extend(new_cards)
        else:
            self.all_cards.append(new_cards)
    def tot(self, act=None):
        total = 0
        for index, card in enumerate(self.all_cards):
            if act is None:
                if total + 11 < 22:
                    total += self.all_cards[index].pairs
                else:
                    if card.rank == 'Ace':
                        card.pairs = 1
                        total += self.all_cards[index].pairs
                    else:
                        total += self.all_cards[index].pairs
            else:
                total += self.all_cards[index].pairs
        return total
    
    def show_all(self):
        print(f"{self.name}'s hand", end=':')
        for index, _ in enumerate(self.all_cards):
            print(f' {self.all_cards[index]} ', end="-")
        print(f' ({self.tot()})')
    
    def __len__(self):
        return len(self.all_cards)
    

In [5]:
class Dealer(Hand):
    def __init__(self, name='Dealer'):
        self.name = name
        self.all_cards = []
    def show_some(self):
        print("Dealer's hand", end=':')
        for index, _ in enumerate(self.all_cards):
            if index == 1:
                continue
            else:
                print(f' {self.all_cards[index]} ', end="-")
        print(f' ( {self.tot() - self.all_cards[1].pairs})')
    
class Player(Hand):
    def __init__(self, name, bank):
        self.name = name
        self.bank = bank
        self.all_cards = []  
        
class Chips:
    def __init__(self):
        self.start = 1500
        self.all = 1500
        self.bet = 10
    def win_bet(self):
        self.all += self.bet
    def lose_bet(self):
        self.all -= self.bet
    def count_chips(self, end):
        self.end = end
        total = self.end - self.start
        t = '\t'
        if total > 0:
            print(f'{t*2}Balance: +{total}')
        else:
            print(f'{t*2}Balance: {total}')
    


In [6]:
def launch_cards():
    new_deck = Deck()
    new_deck.shuffle()
    return new_deck
    
def hit_or_stand():
    choice = ''
    while choice not in ['hit','h', 'stand', 's']:
        choice = input('Hit(h) or Stand(s)?: ').lower()
    clear_output()
    return choice
def hit_dealer(new_deck, dealer):
    if dealer.tot() < 17:
        while dealer.tot() < 17:
            dealer.add_card(new_deck.deal_one())
    else:
        print('Cannot hit')

def player_busted(player, bank):
    bank.lose_bet()
    n = '\n'
    t = '\t'
    print(f'{n}{t*2}{player.name.upper()} LOSES!')
    print(f'{t*2}Bank: {bank.all}')
def player_wins(player, bank):
    n = '\n'
    t = '\t'
    bank.win_bet()
    print(f'{n}{t*2}{player.name.upper()} WINS!')
    print(f'{t*2} Bank: {bank.all}')
def dealer_busted(player, bank):
    n = '\n'
    t = '\t'
    bank.win_bet()
    print(f'{n}{t*2}{player.name.upper()} WINS! Dealer busted')
    print(f'{t*2}Bank: {bank.all}')

def dealer_wins(player, bank):
    n = '\n'
    t = '\t'
    bank.lose_bet()
    print(f'{n}{t*2}{player.name.upper()} LOSES!')
    print(f'{t*2}Bank: {bank.all}')
    
def push(player, bank):
    t = '\t'
    print('\n\t\tPUSH!')
    print(f'{t*2}Bank: {bank.all}')

    
def show_in_game_result(player, dealer):
    player.show_all()
    dealer.show_some()
    
def show_final_result(player, dealer):
    clear_output()
    for hand in [player, dealer]:
        hand.show_all()
    
def check(player, dealer, bank, act):
    if act == 'stand':
        if player.tot() < 22 and dealer.tot() < 22:
            if player.tot() > dealer.tot():
                show_final_result(player, dealer)
                player_wins(player, bank)
            elif dealer.tot() > player.tot():
                show_final_result(player, dealer)
                dealer_wins(player, bank)
            else:
                show_final_result(player, dealer)
                push(player, bank)
        elif player.tot() >= 22:
            show_final_result(player, dealer)
            dealer_wins(player, bank)
        elif dealer.tot() >= 22:
            show_final_result(player, dealer)
            dealer_busted(player, bank)
            
    elif act == 'hit':
        if player.tot() < 22:
            return True
        else:
            show_final_result(player, dealer)
            player_busted(player, bank)
    elif act == 'distribution':
        if player.tot() == 21:
            if dealer.tot() != 21:
                print('\n\t\tBLACK JACK\n')
                player_wins(player, bank)
            else:
                push()
                print('Both got BLACK JACK')
        elif player.tot(act) == 22:
            show_final_result(player, dealer)
            player_busted(player, bank)
        elif player.tot(act) < 21:
            return True
        
def stand(new_deck, player, dealer, bank):
    act = 'stand'
    hit_dealer(new_deck, dealer)
    check(player, dealer, bank, act)
    
def hit (new_deck, player, dealer, bank):
    act = 'hit'
    player.add_card(new_deck.deal_one())
    show_in_game_result(player, dealer)
    playing = check(player, dealer, bank, act)
    return playing
        
def take_bet(text, bank):
    take = True
    while take:
        try:
            bet = int(input(text))
            if bet <= bank.all:
                return bet
            print('You exceed your bank value')
        except ValueError:
            print('Invalid value for bet')
            
def play_again(bank):
    answer =  input('Would you like to play again? ("Yes" or "No")').lower()
    if answer == 'yes':
        if bank.all > 0:
            clear_output()
            return True
        else:
            clear_output()
            print('Your bank is empty. Need to deposit money')
            ready = input('Ready to deposit? ("Yes" or "No"): ').lower()
            if ready == 'yes':
                clear_output()
                deposit = int(input('Deposit: '))
                bank.all = deposit
                return True
            else:
                clear_output()
                print('Ok, game is over')
                return False
    else:
        clear_output()
        print()
        bank.count_chips(bank.all)
        print('\n\t\tG A M E   O V E R\n')
        return False
    
def dealing_two(player, dealer, new_deck):
    for hand in [player, dealer]:
        hand.add_card(new_deck.deal_two())
    
        
def game_on(): 
    game = True
    name = input('Your name: ')
    clear_output()
    bank = Chips()
    while game == True:
        act = 'distribution'
        player = Player(name, bank.all)
        print(f'bank: {bank.all}')
        dealer = Dealer()
        bank.bet = take_bet('Place your bets: ', bank)
        clear_output()
        new_deck = launch_cards()
        dealing_two(player, dealer, new_deck)
        show_in_game_result(player, dealer)
        playing = check(player, dealer, bank, act)
        while playing == True:
            choice = hit_or_stand()
            if choice in ['hit', 'h']:
                playing = hit(new_deck, player, dealer, bank)
            elif choice in ['stand', 's']:
                stand(new_deck, player, dealer, bank)
                playing = False
            
        game = play_again(bank)
        
        

In [7]:
game_on()


		Balance: +1500

		G A M E   O V E R

